In [1]:
# 1. Install the required packages
# On Windows, you just need to execute this cell for once.
try:
    import google.colab
    # IN_COLAB = True
except ImportError:
    # IN_COLAB = False
    %pip install -q git+https://github.com/huggingface/transformers
    %pip install -q git+https://github.com/huggingface/accelerate

%pip install -q git+https://github.com/huggingface/diffusers
%pip install -q gradio ftfy tensorboard
%pip install -q bitsandbytes
#%pip install -U git+https://github.com/TimDettmers/bitsandbytes.git
%pip install -q xformers --index-url https://download.pytorch.org/whl/cu124
#%pip install -U git+https://github.com/facebookresearch/xformers.git@main
print("Package installation finished.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:

In [59]:
# 2. Create folders and download training scripts
import os, shutil

dataset_dir = "./dataset"
output_dir = "./output"
logging_dir = "./logs"

available_models = {
    "SD v1.5": "stable-diffusion-v1-5/stable-diffusion-v1-5",
    # "SDXL": "stabilityai/stable-diffusion-xl-base-1.0"
}

# Create the directories if they don't exist
os.makedirs(dataset_dir, exist_ok=True)
# Delete the 'output' folder and its contents
shutil.rmtree(output_dir, ignore_errors=True)
os.makedirs(output_dir, exist_ok=True)
# Delete the 'log' folder and its contents
shutil.rmtree(logging_dir, ignore_errors=True)
os.makedirs(logging_dir, exist_ok=True)
# Delete the 'dataset' folder and its contents
# shutil.rmtree(dataset_dir, ignore_errors=True)
# os.makedirs(dataset_dir, exist_ok=True)

# fetch textual inversion code if it doesn't exist
if not os.path.exists("textual_inversion.py"):
    !wget https://raw.githubusercontent.com/jomo0825/MrFuGenerativeAI/main/TextualInversion/textual_inversion.py
else:
    print("textual_inversion.py already exists, skipping download.")

ipynb_checkpoints = os.path.join( dataset_dir, ".ipynb_checkpoints")
shutil.rmtree(".gradio", ignore_errors=True)
shutil.rmtree(".config", ignore_errors=True)
shutil.rmtree(ipynb_checkpoints, ignore_errors=True)

textual_inversion.py already exists, skipping download.


In [3]:
# 3. Upload dataset images
def load_dataset():
  try:
    from google.colab import files
    import os

    # Upload files from local machine
    uploaded = files.upload()

    # Move the uploaded files to the target directory
    for filename in uploaded.keys():
        # Get source and destination paths
        source_path = filename
        destination_path = os.path.join(dataset_dir, filename)

        # Move the file
        !mv "{source_path}" "{destination_path}"
        print(f"Moved {filename} to {dataset_dir}")

  except ImportError:
    import tkinter as tk
    from tkinter import filedialog
    import os

    # Initialize tkinter
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    root.attributes('-topmost',True)

    # Open a file dialog and allow multiple file selection
    file_paths = filedialog.askopenfilenames(
        title='Select Dataset Images',
        filetypes=[('Image Files', '*.png;*.jpg;*.jpeg;*.bmp;*.gif')]
    )
    root.destroy()

    # Define the target directory
    target_directory = dataset_dir

    # Copy or move files to the target directory
    for file_path in file_paths:
        filename = os.path.basename(file_path)
        destination = os.path.join(target_directory, filename)
        os.rename(file_path, destination)  # or use shutil.copy for copying
        print(f'File {filename} saved to {target_directory}')

load_dataset()

Saving sks (2).png to sks (2).png
Saving sks (3).png to sks (3).png
Saving sks (4).png to sks (4).png
Saving sks (5).png to sks (5).png
Saving sks (6).png to sks (6).png
Saving sks (1).jpg to sks (1).jpg
Saving sks (7).png to sks (7).png
Moved sks (2).png to ./dataset
Moved sks (3).png to ./dataset
Moved sks (4).png to ./dataset
Moved sks (5).png to ./dataset
Moved sks (6).png to ./dataset
Moved sks (1).jpg to ./dataset
Moved sks (7).png to ./dataset


In [60]:
import gradio as gr
from textual_inversion import main as train_textual_inversion
from textual_inversion import parse_args
# from textual_inversion_sdxl import main as train_textaul_inversion_sdxl
# from textual_inversion_sdxl import parse_args as parse_args_sdxl
import threading, os, logging, time
from os import path
from PIL import Image

def parse_lr_schedule(lr_schedule_str):
    schedule = []
    segments = lr_schedule_str.split(',')
    for segment in segments:
        if ':' in segment:
            lr, steps = segment.split(':')
            schedule.append((float(lr), int(steps)))
        else:
            schedule.append((float(segment), None))  # Final constant learning rate
    return schedule

def get_learning_rate_at_step(lr_schedule, step):
    current_step = 0
    for lr, segment_steps in lr_schedule:
        if segment_steps is None or step < current_step + segment_steps:
            return lr
        current_step += segment_steps
    return lr_schedule[-1][0]  # Return the last LR if beyond defined steps

# Callback to update the preview image in the UI
def preview_callback(image, step):
    global current_preview, current_status, max_train_steps, current_step
    current_step = step
    current_preview = image
    # current_status = f"Preview updated at step {step}"
    # current_preview.show()
    print(f"{step}/{max_train_steps}")


def run_training(model_name, prompt, placeholder_token, initializer_token, num_training_steps,
                 learning_rate, batch_size, preview_save_steps, preview_seed):
    global current_preview, current_status
    global max_train_steps, current_step
    current_preview = None  # Reset the preview
    current_status = "Training started..."  # Initial status

    # Construct the command with all arguments
    command = [
        # "python", "textual_inversion.py",
        "--pretrained_model_name_or_path", available_models[model_name],
        "--train_data_dir", dataset_dir,
        "--placeholder_token", placeholder_token,
        "--initializer_token", initializer_token,
        "--resolution", str(512),
        "--train_batch_size", str(batch_size),
        "--gradient_accumulation_steps", "1",
        "--learning_rate", str(learning_rate),
        "--max_train_steps", str(num_training_steps),
        "--save_steps", str(preview_save_steps),
        "--validation_steps", str(preview_save_steps),
        "--output_dir", output_dir,
        "--logging_dir", logging_dir,
        "--validation_prompt", prompt,
        "--learnable_property", "object",
        "--seed", str(preview_seed),
        "--mixed_precision", "fp16",
        "--variant", "fp16"
        "--enable_xformers_memory_efficient_attention"
    ]

    # Print the command for debugging
    # print("Command:", " ".join(command))

    # Create the directories if they don't exist
    # os.makedirs(logging_dir, exist_ok=True)
    # os.makedirs(output_dir, exist_ok=True)

    # Disable logging
    # logging.getLogger("accelerate").disabled = True

    # Run the command in a separate process
    # process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Run the command as function
    # if model_name == "SDXL":
    #     args = parse_args_sdxl(command)
    # else:
    args = parse_args(command)

    yield gr.update(value=None), gr.update(value=" ".join(command))

    max_train_steps = args.max_train_steps

    def worker(finish_event):
      try:
        train_textual_inversion(args, {"_callback": preview_callback})
      except Exception as e:
        print(f"Training error: {e}")
      finally:
        finish_event.set()

    finish_event = threading.Event()
    finish_event.clear()

    train_thread = threading.Thread(target=worker, args=(finish_event,))
    train_thread.start()
    yield gr.update(value=None), gr.update(value=f"Training started.")

    # except Exception as e:
    #   print(f"Error: {e}")


    while not finish_event.is_set():
        if current_preview is not None:
            yield gr.update(value=current_preview), gr.update(value=f"Preview at {current_step} step.")
            current_preview = None
        time.sleep(1)

    train_thread.join()

    # Print the output and errors (for debugging)
    # print("Output:", stdout.decode())
    # print("Errors:", stderr.decode())

    # Update status when training completes
    current_status = "Training completed!"

    yield gr.update(), gr.update(value=current_status)

def ui():
    with gr.Blocks() as demo:
        gr.Markdown("# Stable Diffusion Textual Inversion UI")
        gr.Markdown("Generate images using a preloaded textual inversion model.")

        with gr.Row():
            model_name = gr.Dropdown(
                label="Model Name",
                choices=available_models.keys(),
                value= list(available_models.keys())[0] if available_models.keys() else "Select a Model",
                interactive=True
            )

        with gr.Row():
            with gr.Column(scale=1, min_width=300):
                placeholder_token = gr.Textbox(label="Placeholder Token", value="sks", placeholder="Enter placeholder token here", interactive=True)
                initializer_token = gr.Textbox(label="Initializer Token", value="girl", placeholder="Enter initializer token here", interactive=True)
                prompt = gr.Textbox(label="Preview Prompt", value="sks", placeholder="Enter your prompt here", interactive=True)
                num_training_steps = gr.Number(label="Number of Training Steps", value=1000, interactive=True)
                learning_rate = gr.Number(label="Learning Rate", value=0.001, interactive=True)
                batch_size = gr.Number(label="Batch Size", value=1, interactive=True)
                preview_save_steps = gr.Number(label="Preview/Save Every N Steps", value=10, interactive=True)
                preview_seed = gr.Number(label="Preview Seed", value=1, interactive=True)
            with gr.Column(scale=1, min_width=300):
                output_image = gr.Image(label="Generated Image")
                generate_status = gr.Textbox(value="Status messages will appear here.", label="Status", interactive=False)
                generate_button = gr.Button("Start Training")


        generate_button.click(
            fn=run_training,
            inputs=[model_name, prompt, placeholder_token, initializer_token, num_training_steps,
                    learning_rate, batch_size, preview_save_steps, preview_seed],
            outputs=[output_image, generate_status],
            show_progress=True,
            queue=True
        )

    return demo

demo = ui()

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://772fc781b5cabde87b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [42]:
demo.close()

Closing server running on port: 7877


In [49]:
!python textual_inversion.py --pretrained_model_name_or_path stable-diffusion-v1-5/stable-diffusion-v1-5 --train_data_dir ./dataset --placeholder_token sks --initializer_token girl --resolution 512 --train_batch_size 1 --gradient_accumulation_steps 1 --learning_rate 0.001 --max_train_steps 100 --save_steps 10 --validation_steps 10 --output_dir ./output --logging_dir ./logs --validation_prompt sks --learnable_property object --seed 1

2025-03-25 17:47:24.101794: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742924844.123407   22765 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742924844.130013   22765 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
03/25/2025 17:47:28 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

tokenizer_config.json: 100% 806/806 [00:00<00:00, 5.67MB/s]
vocab.json: 100% 1.06M/1.06M [00:00<00:00, 6.05MB/s]
merges.txt: 100% 525k/525k [00:00<00:00, 43.8MB/s]
special_tokens_map.json: 100% 472/472 [00:00<00:00, 4.27MB/s]
scheduler_config.json: 100% 30

In [56]:
command = [
    # "python", "textual_inversion.py",
    "--pretrained_model_name_or_path", "stable-diffusion-v1-5/stable-diffusion-v1-5",
    "--train_data_dir", "dataset",
    "--placeholder_token", "sks",
    "--initializer_token", "girl",
    "--resolution", str(512),
    "--train_batch_size", str(1),
    "--gradient_accumulation_steps", "1",
    "--learning_rate", str(0.001),
    "--max_train_steps", str(1000),
    "--save_steps", str(10),
    "--validation_steps", str(10),
    "--output_dir", "output",
    "--logging_dir", "logs",
    "--validation_prompt", "sks",
    "--learnable_property", "object",
    "--seed", str(1),
    "--mixed_precision", "fp16",
    # "--variant", "fp16"
    # "--enable_xformers_memory_efficient_attention"
]

# Print the command for debugging
# print("Command:", " ".join(command))

# Create the directories if they don't exist
# os.makedirs(logging_dir, exist_ok=True)
# os.makedirs(output_dir, exist_ok=True)

# Disable logging
# logging.getLogger("accelerate").disabled = True

# Run the command in a separate process
# process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Run the command as function
# if model_name == "SDXL":
#     args = parse_args_sdxl(command)
# else:
args = parse_args(command)
train_textaul_inversion(args)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
{'clip_sample_range', 'sample_max_value', 'timestep_spacing', 'variance_type', 'thresholding', 'dynamic_thresholding_ratio', 'rescale_betas_zero_snr', 'prediction_type'} was not found in config. Values will be initialized to default values.
{'latents_std', 'force_upcast', 'scaling_factor', 'use_quant_conv', 'mid_block_add_attention', 'use_post_quant_conv', 'shift_factor', 'latents_mean'} was not found in config. Values will be initialized to default values.
All model checkpoint

Steps:   0%|          | 0/1000 [00:00<?, ?it/s]

{'requires_safety_checker', 'image_encoder'} was not found in config. Values will be initialized to default values.


log_validation


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

{'timestep_spacing', 'prediction_type'} was not found in config. Values will be initialized to default values.
Loaded scheduler as PNDMScheduler from `scheduler` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
Loaded feature_extractor as CLIPImageProcessor from `feature_extractor` subfolder of stable-diffusion-v1-5/stable-diffusion-v1-5.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 

KeyboardInterrupt: 